In [1]:
import sys # only for testing
import datetime as dt # only for testing

import pandas as pd
from lark import Lark, Tree, Token
from numpy import random as np_rnd

class LogicLanguage():
       
    def __init__(self, seed=None, resetTree=True):
        # rnd seed:----------------------
        self.seed = seed
        self.rnd_State = np_rnd.RandomState(seed)
        
        # opers grammar:-----------------
        self.oper = self.__init_operators(None) 
        self.boolean_grammar = self.__init_grammar(self.oper)
        self.lk = Lark(self.boolean_grammar, parser='lalr') # PARSE LARK
        
        # vars priority:-----------------
        self.df_priority = None
        self.df_tranClosure = self.__transitive_closure([]) #In presenza di priorità, variabili da cui si può partire
        
        # default and groups
        self.df_defaultVars = None # dataframe, user default variables
        self.groups = None # list of set/list/tuple of values 'bind' together 
        
        # parsed tree:-------------------
        self.allVars = set() # rempio man mano che utente inserisce i nuovi vincoli
        self.parent_parsTree = None # parentNode's parsed tree 
        self.all_parsTree = None # all constrend -> used only in SAVE_SPACE_FLAG == False (all in only one big tree)
        
        # Constraint database:-----------
        self.SAVE_SPACE_FLAG = False # work in profress
        self.db_userConstr = self.DataBase(name='userConstr')
        self.db_treeConstr = self.DataBase(name='treeConstr')
        
        # count tree vars:---------------
        self.CHILDREN_CHOICE_TYPE = 'random'
        self.RESET_TREE_FLAG = resetTree
        self.all_varTree = [None]
        self.iTree, self.countTree, self.runPerTree = 0, 0, -1 #20 -1 -> use only one tree (for feature updates)
        
        # saturate optios -> see function 'set_saturate_options(...)'
        self.saturate_options = {'state':False, 'freeVars':'all', 'values':'random' }
        self.USED_SATURATE_OPTIONS_FLAG = False # used in ricursion
                
        
    #=============================================================================
    def __init_operators(self, oper):
        if oper is None:
            oper = {"AND":'and', 'OR':'or', 'NOT':'not', 'TRUE':'true', 'FALSE':'false'} 
            #print('Will use the following operator as default:')
            #print( ''.join([f'{k} as {v}, ' for k,v in oper.items()])[:-1])
            return oper
        return oper
            
    #-----------------------------------------------------------------------------    
    def __init_grammar(self, oper):
        boolean_grammar = f"""
        ?start: {oper["OR"]}

        ?{oper["OR"]}: {oper["AND"]} 
            | {oper["OR"]} "{oper["OR"]}" {oper["AND"]}   -> {oper["OR"]}

        ?{oper["AND"]}: atom
            | {oper["AND"]} "{oper["AND"]}" atom  -> {oper["AND"]}

        ?atom: "{oper["NOT"]}" atom         -> {oper["NOT"]}
             | constant           -> constant
             | NAME               -> var
             | "(" {oper["OR"]} ")"
             | "[" {oper["OR"]} "]"

        ?constant: "{oper["TRUE"]}"  -> {oper["TRUE"]}
                 | "{oper["FALSE"]}" -> {oper["FALSE"]}
                
        %import common.CNAME -> NAME
        %import common.NUMBER
        %import common.WS_INLINE

        %ignore WS_INLINE
        """
        return boolean_grammar
    
    #=============================================================================
    # ---------------------------- DEBUG PURPOSE --------------------------------
    def parse(self, string): 
        return self.lk.parse(string)
    
    def to_string(self, tree):
        return self.__to_string(tree)
    
    #=============================================================================
    # ------------------------- SET: USER OPTIONS SETTING ------------------------
    def set_saveSpace(self, saveSpace): 
        self.SAVE_SPACE_FLAG = saveSpace # working progress
        
    def set_seed(self, seed=None):
        self.rnd_State = np_rnd.RandomState(seed)
    
    def set_resetTree(self, resetTree):
        self.RESET_TREE_FLAG = resetTree
    
    def set_childrenChoice(self, _type='random'):
        # default -> 'random'
        # 'min_avgLenPath' or 'min_power':
        self.CHILDREN_CHOICE_TYPE = _type
    
    def set_saturate_options(self, state, freeVars='all', values='random'):
        # state: True/False -> use/ don't use the saturate optios
        # freeVars: 'all'/'effective' -> all the variable not assigned are saturate to default/random (paramiter 'values' )
        # values: 'random'/'default' -> ...
       
        # controlli -> work in progress
        self.saturate_options['state'] = state
        self.saturate_options['freeVars'] = freeVars
        self.saturate_options['values'] = values
        
    def set_default(self, default=None):
        # variabili NON in default -> assegnate random
        # defaultVars ->  dict( {var_Name:defautl values}, ...)
        #             -> None: reset default
        self.df_defaultVars = None
        if default is not None:
            try:
                self.df_defaultVars = pd.Series(default).rename('Default')
            except:
                print('The Default format is not correct. Must be:  dict( {var_Name:defautl_values}, ...)', end='')
                print(f"The default_values must be '{self.oper['TRUE']}' or '{self.oper['FALSE']}")
                      
    def set_groups(self, groups):
        # groups -> list of vars name in the same group. ex: list[ (v1,v2),(...), ..., ]
        self.groups = groups
        
    def set_priority(self, priority):
        # From priority (assume DAG) and create a transitive closure
        self.df_priority = pd.DataFrame(priority).sort_values([0,1]).reset_index(drop=True)
        self.df_tranClosure = self.__transitive_closure(priority) 
        
    #=============================================================================
    # ----------------------------- ADD CONSTRAINTS ------------------------------
    def add_constr(self, logic_String, reset_old=False ): 
        if isinstance(logic_String, type('') ) and logic_String == '':
            print('The constraint must be a non empty string')
            return 
    
        if reset_old:
            self.all_parsTree = None
            self.db_userConstr = self.DataBase(name='userConstr')
           
        # parse + possible new Vars + store in DB
        newUserConstr = self.lk.parse(logic_String)
        self.allVars |=  set([t.children[0].value for t in newUserConstr.find_data('var')]) 
        
        if self.SAVE_SPACE_FLAG == True:
            self.db_userConstr.add(newUserConstr) # parse + save tree into the DB
        else:
            if self.all_parsTree is None:
                self.all_parsTree = newUserConstr
            else:
                self.all_parsTree =  Tree(data='and', children=[ newUserConstr, self.all_parsTree])
               
    def reject_Sol(self, solutSet):
        sol=[]
        for k,v in solutSet.items():
            if   v == "true"  : sol.append(k)
            elif v == "false" : sol.append(f'not {k}')
            else: print("wrong paramiter: ignored. Only True or False ")
        return f'not({" and ".join(sol)})'
        
    #=============================================================================
    # ------------------------------ MANAGE DATABASE -----------------------------
    class DataBase(): # working progress

        def __init__(self, name='None', data=None):
            self.database = pd.Series(data=data, name=name, dtype='object')

        def __len__(self):
            return len(self.database)

        def display(self):
            display(self.database)
        
        def remove(self, index):
            # elimino stringa -> mantengo id -> index rimangono IndexRange, else costa di più in termini di spazio
            self.database[index]="" 
            
        def memory_usage(self, index=True, deep=True):
            return self.database.memory_usage(index=index, deep=deep)
        
        # Conversion:--------------------------- 
        def __to_string(self, tree):
            def _to_string(node, df):
                # if Token
                if isinstance(node, type(Token('',''))):
                    df.append( f'K,{node.value},,' )
                    return len(df)-1

                # if Tree
                if isinstance(node, type(Tree('',''))): 
                    m = [_to_string(c, df) for c in node.children] + ['',''] # per caso var che ha 1 o 0 figli    
                    df.append(f'T,{node.data},{m[0]},{m[1]}')
                    return len(df)-1
                
            df = []
            root = _to_string(tree, df)
            return ';'.join(df)
        #---------------------------
        def __to_Tree(self, stringTree):
            def _to_Tree(idNode, dfS):
                dataNode = dfS.iloc[idNode].split(',') 
                # Token()
                if dataNode[0]=='K': return Token('NAME', dataNode[1])
                
                # Tree()
                if dataNode[0]=='T': 
                    m = []
                    for c in dataNode[2:]:
                        if c != '':
                            m += [_to_Tree(int(c), dfS)]
                    return Tree(dataNode[1], m)

            dfS = pd.Series(stringTree.split(';'))
            idRoot = dfS.index[-1]
            return _to_Tree(idRoot, dfS)
       
        # add:---------------------------- 
        def add(self, tree, returnID = True):  
            # Empty database
            newID = 0 if len(self.database)==0 else self.database.index[-1]+1 
            self.database.at[newID] = self.__to_string(tree)
            return newID if returnID else None

        # get Tree: -------------------------
        def get_tree(self, index):
            #return constraint (string)
            return self.__to_Tree(self.database[index])

        def get_trees(self, TreeID_List):
            return [ self.__to_Tree(s) for s in self.database[TreeID_List].values]

        def get_newTree(self, lastID):
            #return all the new constraints in 'AND'
            # assumo nuovi vincoli hanno index_New > index
            return [ self.__to_Tree(s) for s in self.database[lastID+1:].values] # from index(not included) to the end

        def get_all(self):
            #return all the constraints in 'AND'
            return [ self.__to_Tree(s) for s in self.database[:].values]
        
        # get ID: --------------------------
        def get_newTree_ID(self, lastID):
            return list(self.database[lastID+1:].index)

        def get_lastID(self):
            return self.database.index[-1]
        
    #=============================================================================
    # ------------------------------- TREE LARK ----------------------------------
    def reduce_logicStr(self, string, replace={}):
        return self.__to_string(self.reduce_tree( self.lk.parse(string), replace, inplace=True)) #parse->reduct-> toString
        
    #-----------------------------------------------------------------------------
    def __to_string(self, tree):
        # caso base:---------------
        if tree.data == 'var':      return tree.children[0].value
        if tree.data == 'constant': return tree.children[0].data
        if tree.data == 'not':      return f'not({self.__to_string(tree.children[0])})'
        #-----------------------
        # scendo sui rami (prima sx poi dx)
        left, right = self.__to_string(tree.children[0]), self.__to_string(tree.children[1])
        return f'({left} {tree.data} {right})' 
    
    #-----------------------------------------------------------------------------
    def copy_Tree(self, tree):
        if isinstance(tree, type(Token('',''))): 
            return Token(tree.type, tree.value)

        if isinstance(tree, type(Tree('',[]))):    
            newNode = Tree(tree.data,[])
            newNode.children = [self.copy_Tree(c) for c in tree.children]
            return newNode

        print('UNKNOW')
        return None

    def __concat_trees(self, TreeList, inplace=False):  
        # assume there is at least one tree in TreeList
        temp = TreeList[0] if inplace else TreeList[0].copy()
        for t in TreeList[1:]:
            temp = Tree('and',[temp, t if inplace else t.copy() ])
        return temp
    
    #-----------------------------------------------------------------------------
    def isEqual_Tree(self, tree1, tree2): 
        return self.__isEqual_Tree(tree1, tree2)   
    def __isEqual_Tree(self, tree1, tree2):          
        # caso base:---------------
        if tree1.data != tree2.data: return False
        
        if tree1.data == 'var':
            return tree1.children[0].value == tree2.children[0].value # True or False                
        if tree1.data == 'constant':
            return tree1.children[0].data == tree2.children[0].data
        if tree1.data == 'not':
            return self.__isEqual_Tree(tree1.children[0],tree2.children[0])
 
        # recursion :---------------
        left  = self.__isEqual_Tree(tree1.children[0],tree2.children[0])
        if left==False: return False 
        right = self.__isEqual_Tree(tree1.children[1],tree2.children[1])
        return right # if one is false -> return false

    #---------------------------------------------------
    # lark.tree.Tree t -> str
    def reduce_tree(self, root, replace_map={}, inplace=False):
        if inplace:
            rTree = self.__reduce_tree(root, replace_map, inplace)
            root.data, root.children = rTree.data, rTree.children
            return root
        return self.__reduce_tree(root, replace_map, inplace)
    
    def __reduce_tree(self, t, replace_map, inplace):
        # NB: if is necessary enable the inplace options USE the function 'reduce_tree', not this!
        if t.data == 'constant': # true or false
            return t if inplace else Tree('constant', [Tree(t.children[0].data, [])]) #leaf: Tree(data,children)
        #------------------------------
        if t.data == 'var':
            if t.children[0].value in replace_map:
                # converto in nodo in constant
                return Tree('constant', [Tree(replace_map[t.children[0].value], children=[])]) #leaf: Tree(data,children)
            return t if inplace else Tree('var', [Token(t.children[0].type, t.children[0].value)]) #leaf: Token(type,value)     
        #-----------------------------
        if t.data == 'or':
            # left + check if true
            left = self.__reduce_tree(t.children[0], replace_map, inplace)
            if (left.data == 'constant') and (left.children[0].data =='true'): return left # Tree('constant', [Tree('true', children=[])])
            
            # right + check if true
            right = self.__reduce_tree(t.children[1], replace_map, inplace)
            if (right.data == 'constant') and (right.children[0].data == 'true'): return right
                
            if  (left.data == 'constant') and  (left.children[0].data == 'false'):return right
            if (right.data == 'constant') and (right.children[0].data == 'false'):return left  
                
            if inplace:
                t.children = [left, right]
                return t
            return Tree('or', [left, right]) # else new node       
        #-----------------------------
        if t.data == 'and':
            # left + check if true
            left  = self.__reduce_tree(t.children[0], replace_map, inplace)
            if (left.data == 'constant') and (left.children[0].data=='false'): return left
            
            right = self.__reduce_tree(t.children[1], replace_map, inplace)
            if (right.data == 'constant') and (right.children[0].data=='false'): return right
            
            if  (left.data == 'constant') and  (left.children[0].data=='true'): return right
            if (right.data == 'constant') and (right.children[0].data=='true'): return left 
            
            if inplace:
                t.children= [left, right]
                return t
            return Tree('and', [left, right])       
        #-----------------------------
        if t.data == 'not':
            if t.children[0].data == 'not':
                return self.__reduce_tree(t.children[0].children[0], replace_map, inplace)
            
            child = self.__reduce_tree(t.children[0], replace_map, inplace)
            if (child.data == 'constant') and (child.children[0].data=='true'): 
                return Tree('constant', [Tree('false', children=[])])
            if (child.data == 'constant') and (child.children[0].data=='false'): 
                return Tree('constant', [Tree('true', children=[])])
            
            if inplace:
                t.children[0] = child
                return t
            return Tree('not', [child])
        
    #=============================================================================
    # ---------------------------- CHECK SPACE USAGE -----------------------------
    def raw_LarkTree_SpaceUsage(self, tree, sep=' ', tab=0, verbose=False):
        # return the number of bytes
        # size of: class initialization, + data + list of children (NOT children, only pointers are considered) 
        size = sys.getsizeof(tree) 
        if tree is None:
            pass
        
        elif isinstance(tree, type(Tree('',[])) ):
            size += sys.getsizeof(tree.data) + sys.getsizeof(tree.children)
            print(f'{sep*tab}{tree.data}: {size}') if verbose else None

            for c in tree.children:
                size += self.raw_LarkTree_SpaceUsage(c, sep, tab+1, verbose)

        elif isinstance(tree, type(Token('','')) ):
            size += sys.getsizeof(tree.value)
            print(f'{sep*tab}{tree.value}: {size}') if verbose else None
        else:
            print(f'Unknow:{type(tree)}')

        return size  
    # ---------------------------------------
    def raw_myTree_SpaceUsage(self, myTree, sep=' ', tab=0, verbose=False):  
        # return the number of bytes
        # size of: class initialization, + data + list of children (NOT children, only pointers are considered) 
        
        size = sys.getsizeof(myTree)
        keys = sum(map(sys.getsizeof, myTree.keys())) # None, str, int are counted here, But not other structures
        vals = sum(map(sys.getsizeof, myTree.values()))
        print(f'{sep*tab}var:{myTree["v"]}, size:{size}, keys:{keys}, vals:{vals} -> {size+keys+vals}') if verbose else None

        size += keys + vals
        for c in myTree['children']:
            if c is None:
                none = sys.getsizeof(c)
                size += none
                print(f'{sep*tab}none: {none}') if verbose else None
            else:
                res = self.raw_myTree_SpaceUsage(c, sep, tab+1, verbose)
                size += res

        return size
    #=============================================================================
    # ------------------------------ MANAGE PRIORITY -----------------------------
    # funzione ricorsiva per trovare tutti gli i nodi u raggiungibili da v
    def __DFS_Closure(self, startNode, next_Nodes, dfAdj):
        closureList = []
        for u in next_Nodes:
            if (not pd.isna(u)) and (dfAdj.at[u,'visited']==0): #  NON è già stato visitato 
                dfAdj.at[u,'visited'] = 1
                closureList += [(startNode, u)] + self.__DFS_Closure(startNode, dfAdj.at[u,'T'], dfAdj)
        return closureList

    def __transitive_closure(self, edgeList):#edgeList-> list[(v,u),]
        dfEdges = pd.DataFrame(data=edgeList, columns=['S','T']) # Source, Target

        # Find and add leaves + create dataframe Adj
        leaves = set(dfEdges['T'])- set(dfEdges['S'])
        dfEdges = pd.concat([dfEdges, pd.DataFrame(data=leaves, columns=['S'])])
        dfAdj = dfEdges.groupby(['S']).agg({'T':list}).assign(visited = 0)
        
        closureEdgeList = []
        for v in dfAdj.index:
            closureEdgeList += self.__DFS_Closure(v, dfAdj.at[v,'T'], dfAdj)     
            dfAdj['visited'] = 0

        return pd.DataFrame(data=closureEdgeList, columns=['S','T'])     
    
    #----------------------------------------------------------------------------
    def __priority_choice(self, var_list):
        sourceVar = self.df_tranClosure[ self.df_tranClosure['S'].isin(var_list) ]  # tutte i possibili archi uscenti
        targetVar = sourceVar[ sourceVar['T'].isin( sourceVar['S'].tolist() ) ]     # identifico sourceVar che sono target di altre vars
        
        # calcolo differenza dei set
        onlySourceVar = set(sourceVar['S']) - set(targetVar['T'])       
        freeVar = set(var_list) - set(sourceVar['S']).union( set(sourceVar['T']) ) 
        return self.rnd_State.choice( sorted(freeVar.union(onlySourceVar)) )  # sorted to guaranteed riproducibility
    
    #----------------------------------------------------------------------------
    def __side_choice(self, parentNode, _type='random'): 
        
        # 0 == left, 1 == right 
        if _type == 'random':
            return self.rnd_State.choice([0,1], p=[0.5, 0.5])
        
        # if there is no children
        if (parentNode['children'][0] is None) and (parentNode['children'][1] is None):
            return self.rnd_State.choice([0,1], p=[0.5, 0.5])  # default as random
            
        #--------------------------------------
        child_left  = parentNode['children'][0]
        child_rigth = parentNode['children'][1]
        
        avgLenPath_l = child_left['avgLenPath'] if (child_left is not None)  else 0
        avgLenPath_r = child_rigth['avgLenPath'] if (child_rigth is not None) else 0
        
        if _type == 'min_avgLenPath':
            avg_l_normalised =  avgLenPath_l/(avgLenPath_l+avgLenPath_r) 
        elif _type == 'min_power':
            avg_l_normalised = pow(2, avgLenPath_l)/( pow(2, avgLenPath_l) + pow(2, avgLenPath_r) ) 
        else:
            print(f"{_type} not know. Seleced the default opion: random ")
            return self.rnd_State.choice([0,1], p=[0.5, 0.5])  
              
        # favorito quello con la avgLenPath più corta:  0 == left, 1 == right
        return self.rnd_State.choice([0,1], p=[1-avg_l_normalised, avg_l_normalised])      
                
    #=============================================================================
    # --------------------- SAT SOLVER + TREE + NODES ----------------------------  
    def __update_ParentConstr(self, node, assigned, verbose, locations): 
        
        # here, assume at least one value in db_userConstr and every Node has its parent   
        # Evaluate new User Constr
        eval_Trees = []
        newConstr_IDs = self.db_userConstr.get_newTree_ID(node['ID_lastUserConstr'])
        last_ID =  self.db_userConstr.get_lastID()
        
        print(f'(u{locations}-{len(newConstr_IDs)}', end='') if verbose else None
        #print(f'(a{assigned}', end='')
        for newID in newConstr_IDs:
            t = self.db_userConstr.get_tree(newID)
            eval_Trees.append(t)
            
            t = self.reduce_tree(t, assigned) # inplace = False
            # stop evaluation if 'false' is found
            if (t.data == 'constant') and (t.children[0].data=='false'): 
                print('x)', end='') if verbose else None
                return t, last_ID     
            
        print('p', end='') if verbose else None  
        
        # load parent constr
        eval_Trees.append(  self.db_treeConstr.get_tree(node['ID_parentConstr'])  ) # it's a Tree(...)
        newTree = self.reduce_tree( self.__concat_trees(eval_Trees, inplace=True), assigned, inplace=True)    
        print(')', end='') if verbose else None  
        return newTree, last_ID
        
        
    #----------------------------------------------------------------------------
    def sat_Solver(self, predefined={}, verbose=False, test=None): # priority = list of direct edges (v,u)/ test=None/speed/space
        #----------------------------------
        # check existentce of User contrsaints:
        if len(self.db_userConstr)==0 and self.all_parsTree is None:
            print('No constraint inserted')
            return None
    
        # SAT -------------------------------
        if self.RESET_TREE_FLAG:
            self.all_varTree[self.iTree]=None
            self.db_treeConstr = self.DataBase(name='treeConstr')
           
        # check time:----------------------------
        # only for testing purpose
        idSubRun = 0
        start_subRun = None
        end_SubRun = None
        
        res = None  
        while True:
            start_subRun = dt.datetime.now()
            print(f"-->, SunRun:{idSubRun+1}, Started:{start_subRun.strftime('%m-%d %H:%M:%S')}, ", end='') if test is not None else None
            idSubRun += 1
            
           
            if self.countTree == self.runPerTree: # frature in progress 
                self.iTree +=1
                self.countTree = 0
                self.all_varTree += [None]
                
            # reset last parsed for new runs
            self.parent_parsTree = None
            
            # update Root
            if self.all_varTree[self.iTree] is None: 
                self.all_varTree[self.iTree] = {"constr":'', "var":None, "children":[None, None], "isClosed":False, 
                                                'nPath':0, 'avgLenPath':0, 'ID_parentConstr':None, 'ID_lastUserConstr':None}               
                
                if self.SAVE_SPACE_FLAG == True:
                    # We assume exits at leat one constraint inserted by user
                    print('(r',end='') if verbose else None
                    self.parent_parsTree = self.__concat_trees( self.db_userConstr.get_all(), inplace=True) # list[Tree(...)]# reduce incorpored
                    self.parent_parsTree = self.reduce_tree(self.parent_parsTree, predefined, inplace=True)
                    print(')',end='') if verbose else None
                    self.all_varTree[self.iTree]['constr'] = self.parent_parsTree.data # if is already true/flase
                    self.all_varTree[self.iTree]['ID_parentConstr']  = self.db_treeConstr.add(self.parent_parsTree)
                    self.all_varTree[self.iTree]['ID_lastUserConstr'] = self.db_userConstr.get_lastID() 
                else:
                    print('(r',end='') if verbose else None
                    self.parent_parsTree = self.reduce_tree(self.all_parsTree, predefined) # inplace=False
                    self.all_varTree[self.iTree]['constr'] = self.parent_parsTree.data
                    print(')',end='') if verbose else None
                
            # check if root is not closed -> if true => all the nodes are closed            
            if self.all_varTree[self.iTree]['isClosed']:
                print('END', end='') if verbose else None
                break
             
            # resetto eventuale flag di defaul -> per corretta chiusura dei filgi se usata il saturate
            self.USED_SATURATE_OPTIONS_FLAG = False
                       
            # inizio ricerca SAT
            res = self.__step(self.all_varTree[self.iTree], predefined, {}, verbose) 
            
            end_SubRun = dt.datetime.now()
            print(f"runTime:{str(end_SubRun - start_subRun)[:-3]}, ",end='') if test is not None else None
            print(f"spaceDfUser:{self.db_userConstr.memory_usage()}, spaceDfTree:{self.db_treeConstr.memory_usage()}, ", end='') if test =='space' else None
            print(f"spaceVarTree:{self.raw_myTree_SpaceUsage(self.all_varTree[self.iTree])}, spaceConsTree:{self.raw_LarkTree_SpaceUsage(self.all_parsTree)}", end='') if test =='space' else None
            print('') if test is not None else None
            
            print('N') if verbose else None
            if res is not None:
                self.countTree += 1
                break
                
        #---------------------------
        if res is None : return None       # if no solution
        if len(res)==0 : return predefined # len({})=0 if logicStr was riduced to true (false returns None)
        
        # Ad irrelevant variables: not present in res. If doesn' exist an user deafult -> their value will set with random function
        not_inRes = self.allVars - res.keys()  
        if len(not_inRes) != 0:            # sorted x garantire riproducibilità 
            not_inRes = pd.Series(index=sorted(not_inRes), 
                                  data=self.rnd_State.choice(['true', 'false'],len(not_inRes))).to_dict() 
        
            # If does exist an user deafult -> updare the random values
            if self.df_defaultVars is not None:
                not_inRes |= self.df_defaultVars[ list(set(self.df_defaultVars.index) - res.keys()) ].to_dict() 
            return res | not_inRes
        return res
    
    #---------------------------------------------------------
    def __step(self, node, assigedVars, saturatedVars, verbose):
        print('_', end='') if verbose else None
        
        # Update nPath that cross the node
        node['nPath'] += 1
   
        # Check case base -------------------------
        if node['constr'] == 'constant': # Tree reduced to true or false
            node['avgLenPath'] = 1   # set the avgLenPath in a leaf
            
            # check saturateVars' influence: if no -> the node can be closed
            if not self.USED_SATURATE_OPTIONS_FLAG: 
                node['isClosed'] = True
                print('!', end='') if verbose else None

            # check reduce parent_parsTree: true or false----------------- 
            reduced = self.reduce_tree(self.parent_parsTree, assigedVars|saturatedVars, inplace=True) #useless in SAVE_SPACE_FLAG==False
            if reduced.children[0].data == 'true':
                node['constr'] = 'true' 
                print('T', end='') if verbose else None
                return {} | assigedVars | saturatedVars  
            
            elif reduced.children[0].data == 'false':
                node['constr'] = 'false'  
                print('F', end='') if verbose else None
                return None  
            else:
                print(f'PARAMITER UNKNOW: {reduced}', end='') if verbose else None
                node['isClosed'] = True
                return None                 
        #--------------------------------------------
        # if here -> for sure exist a child where continue the research 
        # Assegno Vars -> check if the node has already a var assigned or it's just created
        if node['var'] is None : 
            if self.parent_parsTree is None:
                if self.SAVE_SPACE_FLAG==True:
                    #update parent's constrends with possibly new constrends inserted by user
                    self.parent_parsTree, new_userID = self.__update_ParentConstr(node, assigedVars, verbose, 'V')
                    node['ID_lastUserConstr'] = new_userID
                else:
                    # creo nuovo tree ridotto
                    print('(V',end='') if verbose else None
                    self.parent_parsTree = self.reduce_tree(self.all_parsTree, assigedVars)# inplace = False
                    print(')',end='') if verbose else None
                    
                    
            _vars = self.allVars - assigedVars.keys() - saturatedVars.keys() # rimuovo variabili già assegnate + saturate
            node['var'] = self.__priority_choice(_vars) # scelgo in base alla priorità
        
        #------------------------------------------------------
        # choose one child based on the setted priority (work in progress)
        side = self.__side_choice(node, self.CHILDREN_CHOICE_TYPE) #self.rnd_State.choice([0,1])# 0 == left, 1 == right   
        child, val, newSaturated = self.__next_child( node, side, assigedVars, saturatedVars, verbose)
        
        # if the chosen child is None (it's choose), go to the other child (it must exist for sure)
        if child is None:
            child, val, newSaturated = self.__next_child( node, int(not(side)), assigedVars, saturatedVars, verbose) 
        
        # recorsice call
        res = self.__step(child, assigedVars|val, saturatedVars|newSaturated, verbose) 
        
        #------------------------------------------
        # Update avgLenPath      
        nPath_l, avgLenPath_l = 0,0
        nPath_r, avgLenPath_r = 0,0
        if node['children'][0] is not None:
            nPath_l =  node['children'][0]['nPath'] 
            avgLenPath_l = node['children'][0]['avgLenPath'] 
        if node['children'][1] is not None:
            nPath_r = node['children'][1]['nPath'] 
            avgLenPath_r = node['children'][1]['avgLenPath'] 
            
        node['avgLenPath'] = (nPath_l*avgLenPath_l + nPath_r*avgLenPath_r)/(nPath_l+nPath_r) + 1
        
        
        # Returning sections: ------------------------------------------
        # check if the children both are se closed ->  pruning Tree, to save space
        node["constr"]=''
        if (node['children'][0] is not None) and (node['children'][0]['isClosed']): # check left child 
            if (node['children'][1] is not None) and (node['children'][1]['isClosed']):  # check right child 
                if self.SAVE_SPACE_FLAG==True: 
                    self.db_treeConstr.remove(node['ID_parentConstr'])
                node["isClosed"]=True
                node['children'][0], node['children'][1] = None, None # pruning Tree, to save space

        # return results
        if res is None:
            print('x', end='') if verbose else None
            return None
        else:
            print('^', end='') if verbose else None
            return res    
        
    #----------------------------------------------------------------------------
    def __next_child(self, node, side, assigedVars, saturatedVars, verbose): #side: left or Right

        # check if child exists and if it's closed
        if (node['children'][side] is not None) and (node['children'][side]['isClosed']):
            return None, None, None
        
        # Define val/direction on the tree 
        val = {node['var'] : 'true' if side else 'false'} # 0=false -> left, 1=true -> Right
        
        # check if child exists: No neened to parse -> assign the last checked formula/tree (the parent's one)
        if node['children'][side] is not None:
            return node['children'][side], val, {}
          
        # if here, child doens't exist:---------------------------------
        # Create new child
        node['children'][side] = {"constr":'', "var":None, "children":[None, None], "isClosed":False, 
                                'nPath':0, 'avgLenPath':0, 'ID_parentConstr':None, 'ID_lastUserConstr':None}
           
        if self.SAVE_SPACE_FLAG:
            if self.parent_parsTree is None:
                # if necessary, update parent's constrends with (possibily) new constrends inserted by user
                self.parent_parsTree, new_userID = self.__update_ParentConstr(node, assigedVars|val, verbose, 'C')
                node['children'][side]['ID_lastUserConstr'] = new_userID 
            else:
                # update parent's constrends (inplace!) with only the new assigned vars
                print('(',end='') if verbose else None
                self.reduce_tree(self.parent_parsTree, assigedVars|val, inplace=True)
                print(')',end='') if verbose else None
                node['children'][side]['ID_lastUserConstr'] = node['ID_lastUserConstr']
            
            # save child's parent parsed Tree
            node['children'][side]['ID_parentConstr'] = self.db_treeConstr.add(self.parent_parsTree)
        
        else:
            # riduco stesso tree
            print('(C',end='') if verbose else None
            if self.parent_parsTree is None:
                self.parent_parsTree = self.reduce_tree(self.all_parsTree, assigedVars|val) # inplace=False
            else:
                self.parent_parsTree = self.reduce_tree(self.parent_parsTree, assigedVars|val|saturatedVars, inplace=True)   
            print(')',end='') if verbose else None
        
        # save child's constr parsed Tree
        node['children'][side]['constr'] = self.parent_parsTree.data
        
        # check saturate optios: -----------------------------------------
        newSaturated = {}
        if self.saturate_options['state']==True:
            # reduce parent_parsTree taking in account the saturatedVars (this will be evaluated in the base case of the recursion)
            saturated_Tree = self.reduce_tree(self.parent_parsTree, assigedVars|val|saturatedVars)# inplace=False (inutile in SAVE_SPACE_FLAG==False)
            saturated_Tree, newSaturated = self.__saturate(saturated_Tree, assigedVars|val|saturatedVars, verbose)
            node['children'][side]['constr'] = saturated_Tree.data 
            
            if self.SAVE_SPACE_FLAG == False:
                self.parent_parsTree = saturated_Tree
            
        return node['children'][side], val, newSaturated
    
    #----------------------------------------------------------------------------
    def __saturate(self, saturated_Tree, assigedVars, verbose):
        
        mapDef = self.__saturate_freeVars(assigedVars, self.saturate_options)          
        if len(mapDef)==0: # empty
            return saturated_Tree, {} # continua normale
            
        # check mapDef's influence: (must know for to defined the correct node closure criterio)
        print('(s',end='') if verbose else None
        mapDef_Satur_Tree = self.reduce_tree(saturated_Tree, mapDef) # inplace=False
       
        # if the new mapDef return false -> ingore it
        if mapDef_Satur_Tree.data == 'constant':
            if mapDef_Satur_Tree.children[0].data == 'false':
                print(')',end='') if verbose else None
                return saturated_Tree, {}
        
        # if here: check if there are difference in the two trees. If False -> mapDef influenced the results
        print('e',end='') if verbose else None
        if not self.__isEqual_Tree(saturated_Tree, mapDef_Satur_Tree):
            print(')',end='') if verbose else None
            self.USED_SATURATE_OPTIONS_FLAG = True # means: find true statment with default -> the node doesn't needed to be closed
            return mapDef_Satur_Tree, mapDef
          
        # mapDef didn't interfered
        print(')',end='') if verbose else None
        return saturated_Tree, {} 
    
    #----------------------------------------------------------------------------
    def __saturate_freeVars(self, assigedVars, saturate_options): 
        # saturate_options={'state':True/False, 'freeVars':'all'/'effective', 'values':'random'/default }
                      
        temp_default = None 
        df_assign = pd.Series(assigedVars, dtype='object') # dtype 'object' per evitare warning continui 
        _vars = self.allVars          
        
        # check values optios: ---------------------
        if (self.df_defaultVars is None) or (saturate_options['values']=='random'):
            # create a new 'default' with random variables
            temp_default = pd.Series( index=sorted(_vars), data=self.rnd_State.choice(['true', 'false'],len(_vars)))
        else:
            # saturate_options['values']=='default'
            # add vars without default with random values
            notDef = _vars - set(self.df_defaultVars.index)
            notDef = pd.Series( index=sorted(notDef), data=self.rnd_State.choice(['true', 'false'],len(notDef)))
            temp_default = pd.concat([ self.df_defaultVars, notDef])
        
        # check freeVars optios: ---------------------
        if saturate_options['freeVars']=='all':
            return temp_default[ list( set(temp_default.index)-set(df_assign.index) ) ].to_dict()
                      
        #if here saturate_options['freeVars']=='effective':
        mapDef = {}
        if self.groups is None: # same case of 'freeVars=all'
            return temp_default[ list( set(temp_default.index)-set(df_assign.index) ) ].to_dict()
                    
        for setAtoms in self.groups:
            _isin = df_assign.index.isin(setAtoms) # list of True and False -> false if index is NOT in setAtoms
            if _isin.sum()==0:
                # Nessuna variabile del gruppo già assegnata -> tutte a default
                mapDef |= temp_default[setAtoms].to_dict() 
            else:                    
                # Alcune variabili già assegnate
                diff = df_assign[_isin] != temp_default[ df_assign[_isin].index ]
                if diff.sum() == 0:
                    # no diff -> quelle assegnate sono a default -> tutte a default
                    mapDef |= temp_default[setAtoms].to_dict()          

        # ritorno solo i nuovi assegnamenti a default
        newDef = pd.Series(mapDef, dtype='object')
        return newDef[ list(set(newDef.index)-set(df_assign.index)) ].to_dict()
    

In [2]:
if __name__=='__main__':
    import pandas as pd
    import import_ipynb
    import myFunctions as my 
    
    #logicString= 'not (not (a or (b and (c or false))))'
    #logicString= 'a or ( (b and c) or (d and e))'
    
    myLL = LogicLanguage(seed=None, resetTree = False)
    myLL.add_constr('a or b or c or d or e or f')
    myLL.set_saturate_options(state = True, freeVars='all', values='default')
    #myLL.add_constr('a or b or c')
    #myLL.add_constr('a')

    
    sol = []
    i=0
    while True:
        res = myLL.sat_Solver(predefined={'a':'false'}, verbose=True)
        if res is None:
            break
        
        myLL.add_constr(myLL.reject_Sol(res))
        sol += [res]
        #my.buildTree(myLL.all_varTree[0]).write_svg(f'Test_V6_{i}.svg')
        i += 1
        

    sol = pd.DataFrame(sol, columns=['a','b','c','d','e','f']).sort_values(['a','b','c','d','e','f'])
    #sol = pd.DataFrame(sol, columns=['a','b','c']).sort_values(['a','b','c'])
    print(sol)

    #myLL.all_varTree[0]
    #myLL.parent_parsTree
    #my.buildTree(myLL.all_varTree[0]).write_svg(f'Test_V6_{i}.svg')
    print(myLL.db_treeConstr.display())
    print(myLL.db_userConstr.display())
    

importing Jupyter notebook from myFunctions.ipynb
(r)_()(se)_T^N
__(uV-1p)()(se)_T^^N
__(uC-1p)(se)_!T^^N
___(uV-2p)()(se)_!T^^^N
_(uC-4p)(se)_!T^N
___(uC-2p)(se)_T^^^N
____(uV-1p)()(s)_()_!FxxxxxN
____(uC-0p)(se)_!T^^^^N
_____(uC-1p)_!T^^^^^N
END       a      b      c      d      e      f
7  false  false  false  false   true  false
5  false  false  false   true  false  false
1  false  false  false   true   true  false
6  false  false  false   true   true  false
2  false  false  false   true   true   true
3  false  false   true  false  false  false
0  false  false   true   true  false   true
4  false   true  false  false  false  false


0                            
1                            
2                            
3      T,true,,;T,constant,0,
4      T,true,,;T,constant,0,
5      T,true,,;T,constant,0,
6                            
7                            
8     T,false,,;T,constant,0,
9      T,true,,;T,constant,0,
10     T,true,,;T,constant,0,
Name: treeConstr, dtype: object

None


0    K,a,,;T,var,0,;K,b,,;T,var,2,;T,or,1,3;K,c,,;T...
1    K,a,,;T,var,0,;T,not,1,;K,b,,;T,var,3,;T,not,4...
2    K,a,,;T,var,0,;T,not,1,;K,b,,;T,var,3,;T,not,4...
3    K,a,,;T,var,0,;T,not,1,;K,b,,;T,var,3,;T,not,4...
4    K,a,,;T,var,0,;T,not,1,;K,b,,;T,var,3,;T,not,4...
5    K,a,,;T,var,0,;T,not,1,;K,b,,;T,var,3,;T,and,2...
6    K,a,,;T,var,0,;T,not,1,;K,b,,;T,var,3,;T,not,4...
7    K,a,,;T,var,0,;T,not,1,;K,b,,;T,var,3,;T,not,4...
8    K,a,,;T,var,0,;T,not,1,;K,b,,;T,var,3,;T,not,4...
Name: userConstr, dtype: object

None


In [3]:
if __name__=='__main__':
    import pandas as pd
    import import_ipynb
    import myFunctions as my 

    #logicString= 'not (not (a or (b and (c or false))))'
    #logicString= 'a or ( (b and c) or (d and e))'

    myLL = LogicLanguage(seed=144235, resetTree = True)
    #myLL.add_constr('a or b or c')
    myLL.add_constr('a or b or c or d')

    myLL.db_treeConstr.display()
    myLL.db_userConstr.display()

    sol = []
    while True:
        #print('-'*50)
        res = myLL.sat_Solver(verbose=False)
        if res is None:
            break

        sol += [res]
        myLL.add_constr( myLL.reject_Sol(res) )
        #myLL.db_treeConstr.display()
        #myLL.db_userConstr.display()
        
    #sol = pd.DataFrame(sol, columns=['a','b','c','d','e','f']).sort_values(['a','b','c','d','e','f'])
    sol = pd.DataFrame(sol, columns=['a','b','c','d']).sort_values(['a','b','c','d']).reset_index(drop=True)
    #sol = pd.DataFrame(sol, columns=['a','b','c']).sort_values(['a','b','c']).reset_index(drop=True)
    display(sol)
    
    myLL.db_treeConstr.display()
    myLL.db_userConstr.display()

Series([], Name: treeConstr, dtype: object)

0    K,a,,;T,var,0,;K,b,,;T,var,2,;T,or,1,3;K,c,,;T...
Name: userConstr, dtype: object

,a,b,c,d
0,false,false,false,true
1,false,false,true,false
2,false,false,true,true
3,false,true,false,false
4,false,true,false,true
5,false,true,true,false
6,false,true,true,true
7,true,false,false,false
8,true,false,false,true
9,true,false,true,false


0                            
1                            
2                            
3                            
4     T,false,,;T,constant,0,
5                            
6                            
7     T,false,,;T,constant,0,
8                            
9                            
10                           
11    T,false,,;T,constant,0,
12                           
13    T,false,,;T,constant,0,
14    T,false,,;T,constant,0,
15    T,false,,;T,constant,0,
16    T,false,,;T,constant,0,
17                           
18    T,false,,;T,constant,0,
19                           
20                           
21    T,false,,;T,constant,0,
22    T,false,,;T,constant,0,
23    T,false,,;T,constant,0,
24    T,false,,;T,constant,0,
25                           
26    T,false,,;T,constant,0,
27    T,false,,;T,constant,0,
28                           
29    T,false,,;T,constant,0,
30    T,false,,;T,constant,0,
Name: treeConstr, dtype: object

0     K,a,,;T,var,0,;K,b,,;T,var,2,;T,or,1,3;K,c,,;T...
1     K,c,,;T,var,0,;T,not,1,;K,d,,;T,var,3,;T,and,2...
2     K,a,,;T,var,0,;T,not,1,;K,d,,;T,var,3,;T,not,4...
3     K,c,,;T,var,0,;T,not,1,;K,a,,;T,var,3,;T,not,4...
4     K,d,,;T,var,0,;T,not,1,;K,a,,;T,var,3,;T,and,2...
5     K,a,,;T,var,0,;T,not,1,;K,c,,;T,var,3,;T,and,2...
6     K,a,,;T,var,0,;K,c,,;T,var,2,;T,and,1,3;K,b,,;...
7     K,c,,;T,var,0,;T,not,1,;K,b,,;T,var,3,;T,not,4...
8     K,b,,;T,var,0,;T,not,1,;K,d,,;T,var,3,;T,and,2...
9     K,b,,;T,var,0,;T,not,1,;K,c,,;T,var,3,;T,and,2...
10    K,a,,;T,var,0,;K,b,,;T,var,2,;T,not,3,;T,and,1...
11    K,c,,;T,var,0,;K,d,,;T,var,2,;T,not,3,;T,and,1...
12    K,d,,;T,var,0,;T,not,1,;K,a,,;T,var,3,;T,and,2...
13    K,c,,;T,var,0,;K,d,,;T,var,2,;T,and,1,3;K,a,,;...
14    K,d,,;T,var,0,;T,not,1,;K,c,,;T,var,3,;T,and,2...
15    K,d,,;T,var,0,;K,b,,;T,var,2,;T,and,1,3;K,a,,;...
Name: userConstr, dtype: object

In [4]:
if __name__=='__main__':
    import pandas as pd
    import import_ipynb
    import myFunctions as my 

    myLL = LogicLanguage(seed=144235, resetTree = True)
    myLL.set_saveSpace(False)
    myLL.set_resetTree(False) #True
    
    #myLL.add_constr('a or b or c')
    #myLL.add_constr('a or b or c or d')
    myLL.add_constr('a and b and c and d')
   

    myLL.db_treeConstr.display()
    myLL.db_userConstr.display()

    sol = []
    while True:
        #print('-'*50)
        res = myLL.sat_Solver( predefined={'b':'false','a':'false','c':'false','d':'true'}, verbose=False)
        #res = myLL.sat_Solver( predefined={'b':'false'}, verbose=False)
        if res is None:
            break

        sol += [res]
        myLL.add_constr( myLL.reject_Sol(res) )
        #myLL.db_treeConstr.display()
        #myLL.db_userConstr.display()
        
    #sol = pd.DataFrame(sol, columns=['a','b','c','d','e','f']).sort_values(['a','b','c','d','e','f'])
    sol = pd.DataFrame(sol, columns=['a','b','c','d']).sort_values(['a','b','c','d']).reset_index(drop=True)
    #sol = pd.DataFrame(sol, columns=['a','b','c']).sort_values(['a','b','c']).reset_index(drop=True)
    display(sol)
    
    myLL.db_treeConstr.display()
    myLL.db_userConstr.display()

Series([], Name: treeConstr, dtype: object)

Series([], Name: userConstr, dtype: object)

,a,b,c,d


Series([], Name: treeConstr, dtype: object)

Series([], Name: userConstr, dtype: object)

In [5]:
if __name__=='__main__':
    import pandas as pd
    import import_ipynb
    import myFunctions as my 

    #logicString= 'not (not (a or (b and (c or false))))'
    #logicString= 'a or ( (b and c) or (d and e))'

    myLL = LogicLanguage(seed=144235, resetTree = True)
    myLL.set_saveSpace(False)
    
    #myLL.add_constr('a or b or c')
    myLL.add_constr('a or b or c or d')
   

    myLL.db_treeConstr.display()
    myLL.db_userConstr.display()

    sol = []
    while True:
        #print('-'*50)
        res = myLL.sat_Solver(verbose=False)
        if res is None:
            break

        sol += [res]
        myLL.add_constr( myLL.reject_Sol(res) )
        #myLL.db_treeConstr.display()
        #myLL.db_userConstr.display()
        
    #sol = pd.DataFrame(sol, columns=['a','b','c','d','e','f']).sort_values(['a','b','c','d','e','f'])
    sol = pd.DataFrame(sol, columns=['a','b','c','d']).sort_values(['a','b','c','d']).reset_index(drop=True)
    #sol = pd.DataFrame(sol, columns=['a','b','c']).sort_values(['a','b','c']).reset_index(drop=True)
    display(sol)
    
    myLL.db_treeConstr.display()
    myLL.db_userConstr.display()

Series([], Name: treeConstr, dtype: object)

Series([], Name: userConstr, dtype: object)

,a,b,c,d
0,false,false,false,true
1,false,false,true,false
2,false,false,true,true
3,false,true,false,false
4,false,true,false,true
5,false,true,true,false
6,false,true,true,true
7,true,false,false,false
8,true,false,false,true
9,true,false,true,false


Series([], Name: treeConstr, dtype: object)

Series([], Name: userConstr, dtype: object)

In [6]:
if __name__=='__main__':
    def concat_trees( TreeList):  
        # assume there is at least one tree in TreeList
        temp = TreeList[0] 
        for t in TreeList[1:]:
            temp = Tree('and',[temp, t ])
        return temp
      
    def to_string(self, tree):
        def _to_string(self, node, df):

            # if Token
            if isinstance(node, type(Token('',''))):
                df.append( f'K,{node.value},,' )
                return len(df)-1

            # if Tree
            if isinstance(node, type(Tree('',''))): 
                m = [_to_string(self, c, df) for c in node.children] + ['',''] # per caso var che ha 1 o 0 figli    
                df.append(f'T,{node.data},{m[0]},{m[1]}')
                return len(df)-1

        df = []
        root = _to_string(self, tree, df)
        return ';'.join(df)
    
    def to_Tree(stringTree):
        def _to_Tree(idNode, dfS):
            dataNode = dfS.iloc[idNode].split(',') 
            # Token()
            if dataNode[0]=='K':
                return Token('NAME', dataNode[1])
            # Tree()
            if dataNode[0]=='T': 
                m = []
                for c in dataNode[2:]:
                    if c != '':
                        m += [_to_Tree(int(c), dfS)]
                return Tree(dataNode[1], m)

        dfS = pd.Series(stringTree.split(';'))
        idRoot = dfS.index[-1]
        return _buildTree(idRoot, dfS)
